In [10]:
import pandas as pd
import pickle
from sklearn.feature_extraction.text import CountVectorizer


In [11]:
import os

In [17]:
import re

In [2]:
def one_df():
    pass

In [3]:
with open("./data/114-2015/sbill-114-2015.pkl", "rb") as f:
    b2015 = pickle.load(f)

In [4]:
b2015.info()

<class 'pandas.core.frame.DataFrame'>
Index: 28 entries, hr1191 to sjres8
Data columns (total 5 columns):
json_uri             28 non-null object
txt_uri              28 non-null object
subjects_top_term    28 non-null object
subjects             28 non-null object
text                 28 non-null object
dtypes: object(5)
memory usage: 1.3+ KB


In [5]:
with open("./data/114-2016/sbill-114-2016.pkl", "rb") as f:
    b2016 = pickle.load(f)

In [6]:
b2016.info()

<class 'pandas.core.frame.DataFrame'>
Index: 24 entries, hjres88 to sres377
Data columns (total 5 columns):
json_uri             24 non-null object
txt_uri              24 non-null object
subjects_top_term    24 non-null object
subjects             24 non-null object
text                 24 non-null object
dtypes: object(5)
memory usage: 1.1+ KB


In [7]:
b2015.index = [str(2015)+"s-"+value for value in b2015.index]

# FUNCTION

In [ ]:
#corpus.py
def pre_merge(df, year, chamber):
    df.index = [year+chamber+ "-" + value for value in df.index]
    return df

In [41]:
dfl = [b2015, b2016]

In [25]:
query = "sbill"

In [56]:
import resource_paths as rp

In [60]:
reload(rp)

<module 'resource_paths' from 'resource_paths.py'>

# FUNCTION

In [61]:
def merge_bill_dfs(fps, query):
    year_re = re.compile(r"[0-9]{4}")
    all_dfs = []
    for fp in fps:
        year = year_re.search(fp).group(0)
        with open(fp, "rb") as data_file:
            df = pickle.load(data_file)
        df = pre_merge(df, year, query[0])
        all_dfs.append(df)
        
    return pd.concat(all_dfs)
        
        

In [62]:
sbills_df = merge_bill_dfs(rp.get_all_dfs("sbill"), "sbill")

In [67]:
dbills_df = merge_bill_dfs(rp.get_all_dfs("hbill"), "hbill")

In [131]:
allbilldf = pd.concat([sbills_df, dbills_df])

In [69]:
with open("allsbill.pkl", "wb") as f:
    pickle.dump(sbills_df, f)

In [70]:
with open("allhbill.pkl", "wb") as f:
    pickle.dump(dbills_df, f)

In [72]:
allbilldf.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1798 entries, 2006s-hjres47 to 2012h-sconres56
Data columns (total 5 columns):
json_uri             1798 non-null object
txt_uri              1798 non-null object
subjects_top_term    1787 non-null object
subjects             1798 non-null object
text                 1798 non-null object
dtypes: object(5)
memory usage: 84.3+ KB


In [73]:
with open("allbills.pkl", "wb") as f:
    pickle.dump(allbilldf, f)

In [125]:
main_topics = allbilldf.subjects_top_term.unique().tolist()
#    print(uq + ": "+str(len(allbilldf[allbilldf["subjects_top_term"] == uq])))

In [79]:
all_topics = []
for val in allbilldf.subjects:
    for subj in val:
        all_topics.append(subj)

In [90]:
topics = topics.unique()

In [83]:
type(topics.unique())

numpy.ndarray

In [92]:
from collections import Counter

In [89]:
len(topics.unique())

3400

In [93]:
topicscounter = Counter(all_topics)

In [97]:
topicscounter.most_common(200)

[(u'Congress', 921),
 (u'Legislative rules and procedure', 562),
 (u'House of Representatives', 553),
 (u'Congressional oversight', 505),
 (u'Administrative law and regulatory procedures', 361),
 (u'Economics and public finance', 354),
 (u'Appropriations', 306),
 (u'Government operations and politics', 305),
 (u'Executive agency funding and structure', 298),
 (u'Government trust funds', 291),
 (u'Government information and archives', 287),
 (u'Government studies and investigations', 265),
 (u'Higher education', 248),
 (u'Terrorism', 245),
 (u'Armed forces and national security', 241),
 (u'Small business', 240),
 (u'Law', 218),
 (u'Elementary and secondary education', 217),
 (u'Government employee pay, benefits, personnel management', 212),
 (u'Health', 205),
 (u'Civil actions and liability', 201),
 (u'Research and development', 200),
 (u'Finance and financial sector', 200),
 (u'Public contracts and procurement', 195),
 (u'Labor and employment', 194),
 (u'Congressional reporting require

In [98]:
allkeyset = set(topicscounter.keys())

In [99]:
mainkeyset = set(allbilldf.subjects_top_term.unique())

In [100]:
nottopkeyset = allkeyset-mainkeyset

In [104]:
trimmed_counter = {}
for key in nottopkeyset:
    trimmed_counter[key] = topicscounter[key] 

In [129]:
trimmed_counter.most_common(41)

[(u'Legislative rules and procedure', 562),
 (u'House of Representatives', 553),
 (u'Congressional oversight', 505),
 (u'Administrative law and regulatory procedures', 361),
 (u'Appropriations', 306),
 (u'Executive agency funding and structure', 298),
 (u'Government trust funds', 291),
 (u'Government information and archives', 287),
 (u'Government studies and investigations', 265),
 (u'Higher education', 248),
 (u'Terrorism', 245),
 (u'Small business', 240),
 (u'Elementary and secondary education', 217),
 (u'Government employee pay, benefits, personnel management', 212),
 (u'Civil actions and liability', 201),
 (u'Research and development', 200),
 (u'Public contracts and procurement', 195),
 (u'Congressional reporting requirements', 185),
 (u'Licensing and registrations', 184),
 (u'Department of Defense', 178),
 (u'Administrative remedies', 175),
 (u'Advisory bodies', 172),
 (u'Hazardous wastes and toxic substances', 172),
 (u'Latin America', 171),
 (u'Research administration and fundi

In [130]:
l = [name[0] for name in trimmed_counter.most_common(41)]

In [132]:
allbilldf = allbilldf[sub_in_bill(allbilldf.subjects, l)]

In [115]:
a = set([1, 2 , 3])
b = set([4, 2, 6])
b in a

False

In [116]:
def sub_in_bill(ser, secondary):
    bool_list = []
    for val in ser:
        in_list = False
        for topic in val:
            if topic in secondary:
                in_list=True
        bool_list.append(in_list)
    
    return bool_list
            

In [117]:
allbilldf.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1798 entries, 2006s-hjres47 to 2012h-sconres56
Data columns (total 5 columns):
json_uri             1798 non-null object
txt_uri              1798 non-null object
subjects_top_term    1787 non-null object
subjects             1798 non-null object
text                 1798 non-null object
dtypes: object(5)
memory usage: 84.3+ KB


In [133]:
allbilldf.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1663 entries, 2006s-hr3199 to 2012h-sconres56
Data columns (total 5 columns):
json_uri             1663 non-null object
txt_uri              1663 non-null object
subjects_top_term    1663 non-null object
subjects             1663 non-null object
text                 1663 non-null object
dtypes: object(5)
memory usage: 78.0+ KB


In [134]:
all_topics = main_topics + l 

In [135]:
len(all_topics)

73

In [136]:
with open("topictrimmedbills.pkl", "wb") as f:
    pickle.dump(allbilldf, f)

In [139]:
with open("list-of-topics.pkl", "wb") as f:
    pickle.dump(all_topics, f)